In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from glob import glob
from collections import Counter
from datetime import datetime
import multiprocessing as mp
import csv
import stanza
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)
from more_itertools import sliced

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

tqdm.pandas()

nlp = stanza.Pipeline(lang='en', processors='tokenize', tokenize_batch_size=100)

2024-06-27 20:01:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-27 20:01:12 INFO: Downloaded file to /Users/venkat/stanza_resources/resources.json
2024-06-27 20:01:12 WARNING: Language en package default expects mwt, which has been added
2024-06-27 20:01:12 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

2024-06-27 20:01:12 INFO: Using device: cpu
2024-06-27 20:01:12 INFO: Loading: tokenize
2024-06-27 20:01:12 INFO: Loading: mwt
2024-06-27 20:01:12 INFO: Done loading processors!


In [2]:
dfs = []
for file in glob('../data/raw_data/*.tsv'):
    dfs.append(
        pd.read_csv(
            file, 
            sep='\t', 
            index_col=None, \
            header=0, 
            low_memory=False,
            quotechar='"',
#             dtype = {
#                 'post_id': str,
#                 'comment_id': str,
#                 'parent_id': str,
#                 'raw_comment': str,
#                 'timestamp': float,
#                 'subreddit': str,
#                 'username': str,
#                 'flair': str,
#                 'score': float
#             },
#             on_bad_lines='skip'
        )
    )
    
df = pd.concat(dfs)

df.shape[0], len(dfs)

(14421566, 33)

In [3]:
del dfs
df.head(2)

,post_id,comment_id,parent_id,raw_comment,timestamp,subreddit,username,flair,score,clean_comment
0,t3_10or15q,j6gcw7b,t3_10or15q,Burrow got pressured alot. Didn't like that.,1675048122,bengals,mccurdy3,NaN,476.0,NaN
1,t3_10or15q,j6gd9xk,t1_j6gcw7b,"Yeah, when your second string guard is going up against a DPOY candidate. It's not gonna end well.",1675048261,bengals,mxyztplk33,,114.0,NaN


### Preprocessing

In [4]:
# Timestamp must be float
df.timestamp = pd.to_numeric(df.timestamp,errors='coerce')
df.score = pd.to_numeric(df.score,errors='coerce')

# Drop all rows without post_id, comment_id, raw_comment, timestamp or subreddit
df = df.dropna(subset=['post_id', 'comment_id', 'raw_comment', 'timestamp', 'subreddit'])

# Remove newlines in string columns and deleted comments
df['clean_comment'] = df.raw_comment.apply(lambda x : x.replace('\n', ' '))
df.flair = df.flair.apply(lambda x : str(x).replace('\n', ' '))
df['clean_comment'] = df['clean_comment'].apply(lambda x : x.replace('\t', ' '))
df.flair = df.flair.apply(lambda x : str(x).replace('\t', ' '))
df['clean_comment'] = df['clean_comment'].apply(lambda x : x.replace('\r', ' '))
df.flair = df.flair.apply(lambda x : str(x).replace('\r', ' '))
df = df[df['clean_comment']!="[deleted]"]
df = df[df['clean_comment']!="[removed]"]

# Remove Reddit formatting for URLS and replace with just link text
df['clean_comment'] = df['clean_comment'].apply(lambda x: re.sub(r'\[(.*)\]\((.*)\)', '\g<1>', x, flags=re.IGNORECASE))

# Remove explicit URLs
df['clean_comment'] = df['clean_comment'].apply(lambda x: re.sub(r'http[s]*\S+', 'URL', x, flags=re.IGNORECASE))

In [5]:
# Remove prefix for posts and only select post items
df = df[df['post_id'].str.startswith('t3_')]
df['post_id'] = df['post_id'].apply(lambda x: x[3:])

In [8]:
df.set_index(['post_id', 'comment_id'], drop=False, inplace=True)
df.head()

post_id comment_id   parent_id  \
post_id comment_id                                   
10or15q j6gcw7b     10or15q    j6gcw7b  t3_10or15q   
        j6gd9xk     10or15q    j6gd9xk  t1_j6gcw7b   
        j6gd57u     10or15q    j6gd57u  t1_j6gcw7b   
        j6gjd30     10or15q    j6gjd30  t1_j6gd57u   
        j6hcxd0     10or15q    j6hcxd0  t1_j6gd57u   

                                                                                                                                                                                                                                                                                                                                                                                                      raw_comment  \
post_id comment_id                                                                                                                                                                                                                                                                                                                                                                                                  
10or15q j6gcw7b                                                                                                                                                                                                                                                                                                                                                      Burrow got pressured alot. Didn't like that.   
        j6gd9xk                                                                                                                                                                                                                                                                                                Yeah, when your second string guard is going up against a DPOY candidate. It's not gonna end well.   
        j6gd57u                                                                                                                                                                                                                                                                                                                     3 backups caught up to us. We had a healthy line when we mauled them week 13.   
        j6gjd30                                                                                                                                                                                                                                                                                   We never mauled them. This comment is just as worse as the “burrowhead” and whatever tf that cincy mayor was on   
        j6hcxd0     Chiefs dealt with the same problem in their SB loss to Tampa Bay. Definitely no shame in a 3 point loss on the road with a makeshift OL. Bengals have Burrow and the best WR corps in the NFL on cheap contracts still, along with great coaching. They're in a great position to make another deep run next year, and not just a hollow "they'll be back," like some pretenders of the past.   

                       timestamp subreddit              username flair  score  \
post_id comment_id                                                              
10or15q j6gcw7b     1.675048e+09   bengals              mccurdy3   nan  476.0   
        j6gd9xk     1.675048e+09   bengals            mxyztplk33        114.0   
        j6gd57u     1.675048e+09   bengals        USAesNumeroUno        186.0   
        j6gjd30     1.675051e+09   bengals  Lowered_expectationz   nan   17.0   
        j6hcxd0     1.675071e+09   bengals     Last_Account_Ever          5.0   

                                                                                                                                                                                                                                             

In [7]:
# def tokenize(comments):
#     docs = nlp.stream('\n\n'.join(comments))
#     new_comments = []
#     for doc in docs:
#         ipdb.set_trace()
#         new_comment = ''
#         for sent in doc.sentences:
#             new_comment += '[CLS] ' + ' '.join([token.text for token in sent.tokens]) + ' ' 
#         new_comments.append(new_comment)
#     assert len(new_comments)==len(comments)
#     return new_comments

# tokenized_comments = []
# # Tokenize and add cls tokens
# for chunk in tqdm(sliced(df, 100), total=len(df)//100):
#     comments= chunk['clean_comment'].values.tolist()
#     tokenized_comments.append(tokenize(comments))

In [9]:
gameinfo = pd.read_csv('../data/gameInfo.tsv', sep='\t')
postinfo = pd.read_csv('../data/postInfo.tsv', sep='\t')

### Restrict comments to those where we got post info, and thus game info

In [10]:
df = df[df['post_id'].isin(postinfo['post_id'].unique())]
df.shape

(7458192, 10)

In [11]:
teaminfo = pd.read_csv('../data/nfl_teams.csv')
teaminfo['team_name_short'] = teaminfo['team_name_short'].apply(lambda x: x.lower())

teams = teaminfo['team_name_short'].values.tolist()
subreddits = teaminfo['subreddit'].values.tolist()

teams_to_subreddit = {teams[i]: subreddits[i] for i in range(32)}
subreddit_to_teams = {subreddits[i]: teams[i] for i in range(32)}
team_names_dict = {x: [x] for x in teams}

for x in teams:
    team_names_dict[x].append(teaminfo[teaminfo['team_name_short']==x]['team_id'].values[0])
    team_names_dict[x].append(teaminfo[teaminfo['team_name_short']==x]['team_id_pfr'].values[0])
    # get approx location name? There will be overlaps
    city_name = teaminfo[teaminfo['team_name_short']==x]['team_name'].values[0].lower().replace(x,'').strip()
    
    team_names_dict[x].append(city_name)
    
    team_names_dict[x] = list(set(team_names_dict[x]))

# A fix for washington commanders error prone data
team_names_dict['commanders'].append('redskins')
team_names_dict['washington'] = team_names_dict['commanders']

team_names_dict['buccaneers'].append('bucs')
team_names_dict['jaguars'].append('jags')
team_names_dict['patriots'].append('pats')
team_names_dict['eagles'].append('philly')
team_names_dict['colts'].append('indiana')
team_names_dict['colts'].append('dolts')
team_names_dict['dolphins'].append('phins')
team_names_dict['chargers'].append('bolts')

# df['team'] = df['subreddit'].apply(lambda x: subreddit_to_team(x))

Figure out the opponent team and cache it in an array for now. Do it per thread to save time?

## Add gametime and win prob and opp columns

In [12]:
df.shape

(7458192, 10)

In [14]:
pbp = pd.read_csv('play_by_play.tsv', sep='\t', low_memory=False)

def get_utc_timestamp(row):
    tod = row['time_of_day']
    date = row['game_date']
    
    day = int(date.split('-')[2])
    month = int(date.split('-')[1])
    year = int(date.split('-')[0])
    
    hour = int(tod.split(':')[0])
    minute = int(tod.split(':')[1])
    second = int(tod.split(':')[2])
    
    # UTC time for NFL games is almost always afternoon? The earliest local time is around 11am, which would be at least 3pm in UTC
    if int(hour)<12: 
        day_obj = dt.date(year, month, day)
        day = (day_obj + dt.timedelta(days=1)).day
    
    timestamp = dt.datetime(year, month, day, hour, minute, second, tzinfo=dt.timezone.utc)
    return int(timestamp.timestamp())

In [15]:
def get_win_prob(row, game_pbp, is_away_team):
    wp = 0
    if row.gametime <= 0.0:
        wp = game_pbp.vegas_home_wp.values[0]
    elif row.gametime >= 1.0:
        wp = game_pbp.vegas_home_wp.values[-1]
    else:
        wp = game_pbp[game_pbp.timestamp<row.timestamp].tail(1).to_dict(orient='records')[0]['vegas_home_wp']
    if is_away_team:
        wp = 1 - wp
    return np.round(wp, 3)

def get_gametime(row, game_pbp):
    game_start = game_pbp.timestamp.min()
    game_end = game_pbp.timestamp.max()
    if row.timestamp<=game_start:
        return 0.0
    elif row.timestamp >= game_end:
        return 1.0
    else:
        return np.round((row.timestamp-game_start)/(game_end-game_start),3)

In [16]:
df['opp'] = ""
df['gametime'] = 0.0
df['win_prob'] = 0.0

for pid in tqdm(df.post_id.unique()):

    # Get PBP for that game
    gameid = postinfo.loc[postinfo.post_id==pid, 'game_id'].values[0]
    game_pbp = pbp.loc[pbp['new_game_id']==gameid, ['game_date','time_of_day','posteam', 'home_team', 'away_team', 'desc', 'vegas_home_wp']].reset_index(drop=True)
    game_pbp = game_pbp.bfill().ffill()
    game_pbp['timestamp'] = game_pbp.apply(lambda x: get_utc_timestamp(x), axis=1)
    game_pbp.sort_values('timestamp', ignore_index=True, inplace=True)
    
    in_team = subreddit_to_teams[df[df['post_id']==pid]['subreddit'].values[0]]
    home_team_name = teaminfo.loc[teaminfo['team_id']==game_pbp['home_team'].values[0], 'team_name_short'].values[0]
    
    # Set opponent column
    if home_team_name!=in_team:
        opp = game_pbp['away_team'].values[0]
    else:
        opp = game_pbp['home_team'].values[0]
    df.loc[df['post_id']==pid, 'opp'] = teaminfo.loc[teaminfo['team_id']==opp, 'team_name_short'].values[0]
    
    # Pre-game threads
    if postinfo[postinfo['post_id']==pid]['type'].values[0]=='pre':
        df.loc[df['post_id']==pid, 'gametime'] =  0.0
        if in_team==home_team_name:
            df.loc[df['post_id']==pid, 'win_prob'] =  np.round(game_pbp.vegas_home_wp.values[0], 3)
        else:
            df.loc[df['post_id']==pid, 'win_prob'] =  np.round(1-game_pbp.vegas_home_wp.values[0],3)
    
    # Post-game threads
    elif postinfo.loc[postinfo['post_id']==pid]['type'].values[0]=='post':
        df.loc[df['post_id']==pid, 'gametime'] =  1.0
        if in_team==home_team_name:
            df.loc[df['post_id']==pid, 'win_prob'] =  np.round(game_pbp.vegas_home_wp.values[-1], 3)
        else:
            df.loc[df['post_id']==pid, 'win_prob'] =  np.round(1-game_pbp.vegas_home_wp.values[-1],3)
    
    # Game threads
    else:
        new_df = df.loc[df['post_id']==pid]
        new_df['gametime'] = new_df.apply(lambda x: get_gametime(x, game_pbp), axis=1)
        df.loc[df['post_id']==pid, 'gametime'] = new_df['gametime']
        df.loc[df['post_id']==pid, 'win_prob'] = new_df.apply(lambda x: get_win_prob(x, game_pbp, in_team==home_team_name), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2405/2405 [1:22:49<00:00,  2.07s/it]


## Write to files

In [17]:
post_types = dict(zip(postinfo['post_id'], postinfo['type']))
df['post_type'] = df['post_id'].apply(lambda x: post_types[x])
df.rename(columns={'score': 'votes'}, inplace=True)

In [18]:
df[df['post_type']=='pre'].to_csv('../data/pre_comments.tsv', sep='\t', columns=['post_id', 'comment_id', 'parent_id', 'clean_comment', 'timestamp', 'subreddit', 'username', 'flair', 'votes', 'opp', 'win_prob', 'gametime'], index=False, escapechar="\\", quoting=csv.QUOTE_NONE)

In [19]:
df[df['post_type']=='post'].to_csv('../data/post_comments.tsv', sep='\t', columns=['post_id', 'comment_id', 'parent_id', 'clean_comment','timestamp', 'subreddit', 'username', 'flair', 'votes', 'opp', 'win_prob', 'gametime'], index=False, escapechar="\\", quoting=csv.QUOTE_NONE)

In [20]:
df[df['post_type']=='game'].to_csv('../data/game_comments.tsv', sep='\t', columns=['post_id', 'comment_id', 'parent_id', 'clean_comment', 'timestamp', 'subreddit', 'username', 'flair', 'votes', 'opp', 'win_prob', 'gametime'], index=False, escapechar="\\", quoting=csv.QUOTE_NONE)

In [21]:
gameinfo.game_id.unique().shape

(569,)

In [22]:
postinfo.game_id.unique().shape

(569,)

In [23]:
postinfo[postinfo.type=='game'].shape

(1104, 4)